In [1]:
# dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [2]:
sumo_results_df = pd.DataFrame()

In [3]:
# read in csv files and create dataframe
for num in range(1983,2019):
    df = pd.read_csv("sumo_results/" +str(num)+".csv")
    sumo_results_df = sumo_results_df.append(df)


In [4]:
sumo_results_df.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),False,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),True
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),True,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),False
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),True,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),False
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),False,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),True
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),False,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),True


In [5]:
# Pull out only kimarite column
kimarite = sumo_results_df[['kimarite']]

In [6]:
kimarite.head()

,kimarite
0,yorikiri
1,yorikiri
2,oshidashi
3,oshidashi
4,yorikiri


In [7]:
# Pull only unique values
kimarite_less = kimarite.drop_duplicates()


In [8]:
# Reset the index
kimarite_less = kimarite_less.reset_index(drop=True)
kimarite_less.head()

,kimarite
0,yorikiri
1,oshidashi
2,tsukidashi
3,hatakikomi
4,yoritaoshi


In [9]:
# Drop/reset index to use new index as new column
kimarite_keys = kimarite_less.reset_index(level=0)

In [10]:
kimarite_keys.head()

,index,kimarite
0,0,yorikiri
1,1,oshidashi
2,2,tsukidashi
3,3,hatakikomi
4,4,yoritaoshi


In [11]:
# Rename column to kimarite_id to create key for relational database
kimarite_keys = kimarite_keys.rename(columns={'index':'kimarite_id'})
kimarite_keys.head()

,kimarite_id,kimarite
0,0,yorikiri
1,1,oshidashi
2,2,tsukidashi
3,3,hatakikomi
4,4,yoritaoshi


In [12]:
# Wikipdeia article with kimarite descriptions and groupings
url='https://en.wikipedia.org/wiki/Kimarite'
res = requests.get(url)

In [13]:
# BeautifulSoup
soup = bs(res.text, 'html.parser')

In [14]:
# Dataframe for Kimarite name, description and type
kimarite_desc = pd.DataFrame()

In [15]:
# Pull h3/h2/p elements
tags = soup.find_all(['h2','h3','p'])
# Find first h2 element
heading = soup.find_all('h2')[1]
heading

<h2><span id="Kihonwaza_.E5.9F.BA.E6.9C.AC.E6.8A.80"></span><span class="mw-headline" id="Kihonwaza_基本技"><i>Kihonwaza</i> 基本技</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=Kimarite&amp;action=edit&amp;section=1" title="Edit section: Kihonwaza 基本技">edit</a><span class="mw-editsection-bracket">]</span></span></h2>

In [32]:
type = heading.find(class_='mw-headline').text

In [17]:
for line in tags:
    print(line.name)

p
p
p
h2
h2
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h2
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h2
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
p
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h2
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h2
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h2
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h3
p
h2
p
h3
p
h3
p
h3
p
h3
p
h3
p
h2
h2
h2
h3
h3
h3
h3
h3
h3
h3
h3
h3
h3
h3


In [36]:
# For loop
next = heading.next_sibling.next_sibling
row = 0

# starting at row=0
if next.name == 'h3':
    # h3 append to kimarite['type']
    kimarite['type'][row] = next.find(class_='mw-headline').text
    next = next.next_sibling
    # skip h3 paragraph desc
    if next.name == 'p':
        next = next.next_sibling
    else next = next.next_sibling
elif next.name == 'h2':
    # If h2, append to kimarite['kimarite'], then append next sibling to kimarite['description']


##### think about just looping through list of items pulled _tags_ to aviod weird next_sibling/comma/paragraph issues

'p'

In [37]:
tags

[<p><i><b>Kimarite</b></i><span style="font-weight: normal"> (<span lang="ja">決まり手</span><span style="display:none">,</span> <i>kimari-te</i>)</span> are winning techniques in a <a href="/wiki/Sumo" title="Sumo">sumo</a> bout. For each bout in a Grand Sumo tournament (or <i><a href="/wiki/Honbasho" title="Honbasho">honbasho</a></i>), a sumo referee, or <i><a href="/wiki/Gy%C5%8Dji" title="Gyōji">gyōji</a></i>, will decide and announce the type of <i>kimarite</i> used by the winner. It is possible (although rare) for the <a href="/wiki/Judge_(sumo)" title="Judge (sumo)">judges</a> to modify this decision later. Records of the <i>kimarite</i> are kept and statistical information on the preferred techniques of different wrestlers can be deduced easily. For example, a <a href="/wiki/Pie_chart" title="Pie chart">pie chart</a> of the <i>kimarite</i> used by each <i><a href="/wiki/Sekitori" title="Sekitori">sekitori</a></i> in the past year can be found on the <a href="/wiki/Japan_Sumo_Associ

In [16]:
# Get first h3
# For loop
p_tags = []
next = firstElem.nextSilbing
while next.name != "h2":
  p_tags.append(next)
  next = next.nextSibling

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Kimarite - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Kimarite","wgTitle":"Kimarite","wgCurRevisionId":883191223,"wgRevisionId":883191223,"wgArticleId":2773330,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles containing Japanese-language text","Sumo terminology"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","Ju